In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Custom Signature with Type Annotation Support

This notebook demonstrates:
- Compact field parser using <|field|> format (57% fewer tokens than DSPy)
- Signature system with metaclass that extracts type annotations
- Type-aware prompt generation and parsing

In [7]:
import re
from typing import Any

class CompactFieldParser:
    """Minimal field parser using <|field|> format."""
    
    PATTERN = re.compile(r'<\|(\w+)\|>')
    
    @staticmethod
    def format(fields: dict[str, Any]) -> str:
        parts = []
        for field_name, value in fields.items():
            parts.append(f"<|{field_name}|>\n{value}")
        return "\n\n".join(parts)
    
    @staticmethod
    def parse(text: str, expected_fields: list[str]) -> dict[str, str]:
        sections = [(None, [])]
        
        for line in text.splitlines():
            match = CompactFieldParser.PATTERN.match(line.strip())
            if match:
                header = match.group(1)
                remaining_content = line[match.end():].strip()
                sections.append((header, [remaining_content] if remaining_content else []))
            else:
                sections[-1][1].append(line)
        
        sections = [(k, "\n".join(v).strip()) for k, v in sections]
        
        fields = {}
        for field_name, content in sections:
            if field_name and field_name not in fields and field_name in expected_fields:
                fields[field_name] = content
        
        return fields

In [11]:
from dataclasses import dataclass
from typing import Dict, Type, Any

@dataclass
class FieldInfo:
    name: str
    type: Type = str
    desc: str = ""
    prefix: str = ""
    is_input: bool = True
    
    def __post_init__(self):
        if not self.prefix:
            self.prefix = self._infer_prefix(self.name)
    
    @staticmethod
    def _infer_prefix(name: str) -> str:
        return name.replace("_", " ").title()

def InputField(desc: str = "", prefix: str = "") -> Any:
    return FieldInfo(name="", desc=desc, prefix=prefix, is_input=True)

def OutputField(desc: str = "", prefix: str = "") -> Any:
    return FieldInfo(name="", desc=desc, prefix=prefix, is_input=False)

class SignatureMeta(type):
    def __new__(mcs, name, bases, namespace, **kwargs):
        input_fields = {}
        output_fields = {}
        
        # Get type annotations
        annotations = namespace.get('__annotations__', {})
        
        for field_name, field_value in list(namespace.items()):
            if isinstance(field_value, FieldInfo):
                field_value.name = field_name
                
                # Extract type from annotation
                if field_name in annotations:
                    field_value.type = annotations[field_name]
                
                if field_value.is_input:
                    input_fields[field_name] = field_value
                else:
                    output_fields[field_name] = field_value
                
                del namespace[field_name]
        
        namespace['_input_fields'] = input_fields
        namespace['_output_fields'] = output_fields
        namespace['_instructions'] = (namespace.get('__doc__', '') or '').strip()
        
        return super().__new__(mcs, name, bases, namespace)
    
    @property
    def input_fields(cls):
        return cls._input_fields
    
    @property
    def output_fields(cls):
        return cls._output_fields
    
    @property
    def instructions(cls):
        return cls._instructions

class Signature(metaclass=SignatureMeta):
    _input_fields = {}
    _output_fields = {}
    _instructions = ""

In [12]:
# Test Type Annotations
class ThisIdeaOfDataType(Signature):
    """A signature representing 'ThisIdeaOfDataType'."""
    field: int = InputField(desc="An example field")

print("Testing type annotation extraction:")
print("Input fields:", ThisIdeaOfDataType.input_fields)
print()
print("Field type:", ThisIdeaOfDataType.input_fields['field'].type)
print("Is it int?", ThisIdeaOfDataType.input_fields['field'].type == int)

Testing type annotation extraction:
Input fields: {'field': FieldInfo(name='field', type=<class 'int'>, desc='An example field', prefix='', is_input=True)}

Field type: <class 'int'>
Is it int? True


In [13]:
# Complete Example
class Calculator(Signature):
    """Solve math problems."""
    expression: str = InputField(desc="Math expression")
    result: int = OutputField(desc="The result")
    explanation: str = OutputField(desc="Brief explanation")

print("=== Calculator Signature ===")
print("Input fields:", Calculator.input_fields)
print("Output fields:", Calculator.output_fields)

=== Calculator Signature ===
Input fields: {'expression': FieldInfo(name='expression', type=<class 'str'>, desc='Math expression', prefix='', is_input=True)}
Output fields: {'result': FieldInfo(name='result', type=<class 'int'>, desc='The result', prefix='', is_input=False), 'explanation': FieldInfo(name='explanation', type=<class 'str'>, desc='Brief explanation', prefix='', is_input=False)}
